# Download ERA5 data 

In [1]:
import os
from IPython.display import HTML
from IPython.display import display
import api_func
from datetime import datetime, timedelta
from netCDF4 import Dataset
import numpy as np
import plotly.express as px
import xarray as xr

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## Pre-requirements


In order to download ERA5 data using an API, you will have to get a UDI and a personal key. To get those, you will have to register to the Copernicus platform using the following link : https://cds.climate.copernicus.eu/user/login
<br>Enter/copy paste your UDI and your key in the cells below

In [ ]:
udi = '83f26bcf-91c9-4570-bfbc-aceb220f80a7'
key = 'e6487f45-a948-4134-b1e0-4d8087f87c73' 

Below you can add a filename after your variable names (eg, `filename = 'ERA5'` will give you __u10_ERA5__).
<br>
By default, files will be downloaded in your home directory, in the subdirectory __api__ , but you can change it through the variable __path__ (otherwise let default value `path = 'None'`).

In [2]:
filename = 'test_final'
path = '/home/kumarv/renga/Documents/Fall_24/API-ERA5/Data'

In [4]:
os.listdir('/home/kumarv/renga/Documents/Fall_24/API-ERA5/Data')

['test_final.nc']

## Variable, time period and geographical location

ERA5 provides hourly estimates for a large number of ocean-wave and atmospheric variables (see list [here](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview)). Variable names must be formatted with lowercase first letters, and composed words spaced by underscores (eg, Sea surface temperature in `'sea_surface_temperature'`, 10m u-component of wind in `'10m_u_component_of_wind'`). Use the alias `'all'` to select all available months, days or hours, otherwise set one or several elements as a list (eg, `'days = ['01','02']'`).

In [5]:
# CHOOSE WHICH SINGLE LEVELS TO DOWNLOAD IN data
variables = [
        "2m_dewpoint_temperature",
        "2m_temperature",
        "total_precipitation",
        "surface_solar_radiation_downward_clear_sky",
        "surface_solar_radiation_downwards",
        "high_vegetation_cover",
        "leaf_area_index_high_vegetation",
        "leaf_area_index_low_vegetation",
        "low_vegetation_cover",
        "type_of_high_vegetation",
        "type_of_low_vegetation",
        "land_sea_mask"
    ]

years = ['2000'] 

months = ['01'] #'all'
#Or set one or several months as a list
# ['01','02','03','04','05','06','07','08','09','10','11','12']

days = ['01', '02', '03','04','05','06','07'] # 'all' #['01'] #'all'
#Or set one or several days as a list
# ['01', '02', 03','04','05','06','07','08','09','10','11','12','13','14','15','16','17',
                   #'18','19','20','21','22','23','24','25','26','27','28','29','30','31']

hours = 'all'
#Or set one or several hours as a list
#['00:00','01:00','02:00','03:00','04:00', '05:00','06:00','07:00','08:00',
                     #'09:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00',
                     #'20:00','21:00','22:00','23:00']

# Boundaries of the geographical area in decimal degrees
north_boundary = 90
south_boundary = -90
east_boundary = 180
west_boundary = -180


## Print summary and run download

In [6]:
api_func.make_cds_file(key, path)

In [ ]:
api_func.return_cdsbeta(filename, variables, years, months, days, hours,  [north_boundary, west_boundary, south_boundary, east_boundary])

You have selected : 

   - 2m_dewpoint_temperature
   - 2m_temperature
   - total_precipitation
   - surface_solar_radiation_downward_clear_sky
   - surface_solar_radiation_downwards
   - high_vegetation_cover
   - leaf_area_index_high_vegetation
   - leaf_area_index_low_vegetation
   - low_vegetation_cover
   - type_of_high_vegetation
   - type_of_low_vegetation
   - land_sea_mask

for the following times
Years : ['2000'] 
 Months : ['01'] 
 Days : all 
 Hours : all

Your boundaries are : North 90°, South -90°, East 180°, West -180°


2024-10-13 18:09:32,749 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-10-13 18:09:32,750 WARNING [2024-09-26T00:00:00] Should you have not yet migrated from the old CDS system to the new CDS, please check our [informative page](https://confluence.ecmwf.int/x/uINmFw) for guidance.
2024-10-13 18:09:32,751 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-10-13 18:09:32,751 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-10-13 18:09:32,752 WARNING [2024-06-16T

eb453b3b8068c6a8ae192dff713239ff.nc:   0%|          | 0.00/4.58G [00:00<?, ?B/s]

### Code to visualize your variables

In [ ]:
os.listdir(path)

In [ ]:
ds = xr.open_dataset(os.path.join(path,filename+'.nc'))
ds

In [ ]:
# Select the variables of interest
variables = ['2m_temperature', '2m_dewpoint_temperature', 'surface_solar_radiation_downwards', 'total_precipitation']
selected_ds = ds[variables]

# Perform daily average for temperature, dewpoint, and solar radiation
daily_avg = selected_ds[['2m_temperature', '2m_dewpoint_temperature', 'surface_solar_radiation_downwards']].resample(time='D').mean()

# Perform daily sum for precipitation
daily_sum_precip = selected_ds['total_precipitation'].resample(time='D').sum()

# Combine the results
result = xr.merge([daily_avg, daily_sum_precip])

# Display the result
print(result)

In [ ]:
import sys
variable_name= 'u10'

try :
    nc = Dataset(os.path.join(path,filename+'.nc'), 'r')
    ds = xr.open_dataset(xr.backends.NetCDF4DataStore(nc))
    print(ds)
except FileNotFoundError :  
    print('no ERA data with this variable')
    sys.exit()

ds = ds[variable_name]

print(ds)



In [ ]:
fig = px.imshow(ds, animation_frame='valid_time', zmin=float(ds.min()), zmax=float(ds.max()), color_continuous_scale='RdBu_r')

fig.show()